# import

In [34]:
import cv2
import os
import os.path as osp
from glob import glob
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np

# path

In [35]:
data_name = 'player'
input_dir = osp.join('../input', data_name)
data_dir = osp.join('../output', data_name, 'tracking')
label_save_dir = osp.join('../output', data_name, 'label')
reid_dir = osp.join('../output', data_name, 'reid')
track_label_path = osp.join('../output/')
movie_dir = osp.join('../output', data_name, 'movie')

# function

In [36]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [37]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

# label save

In [ ]:
games = listdirs(data_dir, 'game')
columns = ['tracking_id', 'frame_index', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'image_name']

for game in games:
    game_dir = osp.join(data_dir, game)
    clips = listdirs(game_dir, 'Clip')

    for clip in tqdm(clips):
        clip_dir = osp.join(game_dir, clip)
        tracking_ids = listdirs(clip_dir)
        labels = []
        for tracking_id in tracking_ids:
            tracking_dir = osp.join(clip_dir, tracking_id)
            label_dir = osp.join(tracking_dir, 'label')
            
            label_names = listdirs(label_dir)
            labels_arr = np.zeros((len(label_names), 7))
            for num, label_name in enumerate(label_names):
                file_name = osp.splitext(label_name)[0]
                label_path = osp.join(label_dir, label_name)
                label = load_pickle(label_path)
                labels.append(label)
        label_df = pd.DataFrame(labels, columns=columns)
        label_df['width'] = label_df['xmax'] - label_df['xmin']
        label_df['height'] = label_df['ymax'] - label_df['ymin']
        
        label_path = osp.join(label_save_dir, game, '{}.csv'.format(clip))
        os.makedirs(osp.dirname(label_path), exist_ok=True)
        label_df.to_csv(label_path, index=False)